# Dataset processing: take the original games.csv, filter then split into 3 files for training, testing, and validation

In [4]:
# Load dataset and filter out some columns
# Original:  https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data

import json 
import csv
import pandas as pd 
import numpy as np
from dateutil import parser

# Load dataset
file_path = "games.csv" 
df = pd.read_csv(file_path)
df.head()

# Keep only the following columns:
keep_columns = ["AppID", "Name", "Release date", "Estimated owners", "Price", "Metacritic score", "Recommendations", "Positive", "Negative", "Publishers", "Genres" ]

# Drop columns that are not in the keep_columns list
df = df[keep_columns]
df = df.dropna()

# For each row that has any missing value, remove it probability 0.8
# How to use probability of 0.8 for each row?
# Drop rows with missing values based on 80% probability
#df = df[~(df.isnull().any(axis=1) & (np.random.rand(len(df)) < 0.8))]

# Remove those with Price smaller than a small epsilon value
# We don't want free games!
df = df[df["Price"] > 0.01]

# Drop the rows with missing values for Metacritic score and User score or where those values are very small to 0
# We don't want games with no score or very low scores
df = df.dropna(subset=["Metacritic score"])
df = df[(df["Metacritic score"] > 0.1)]

# Display the result
#print(df.head())  # Show the first few rows

# Fix some Release date issues

def fix_release_date_format(df: pd.DataFrame):
    """ Convert Release date column to format 'Month Day, Year' """
    def parse_date(date):
        try:
            return parser.parse(date).strftime("%B %d, %Y")  # Format: Month Day, Year
        except (ValueError, TypeError):  # Handle invalid/missing dates
            return None  # Or return "Unknown" if you prefer
    
    df["Release date"] = df["Release date"].apply(parse_date)
    return df

df = fix_release_date_format(df)
df = df.dropna(subset=["Release date"])


# Save to a new file (optional)
df.to_csv("filtered_games.csv", index=False)

In [5]:
df

,AppID,Name,Release date,Estimated owners,Price,Metacritic score,Recommendations,Positive,Negative,Publishers,Genres
10,1026420,WARSAW,"October 02, 2019",20000 - 50000,23.99,62,427,589,212,"Pixelated Milk,gaming company","Indie,RPG"
15,22670,Alien Breed 3: Descent,"November 17, 2010",200000 - 500000,9.99,64,285,349,134,Team17 Digital Ltd,Action
42,231330,Deadfall Adventures,"November 15, 2013",100000 - 200000,19.99,53,1140,1716,628,THQ Nordic,"Action,Adventure"
44,897820,Reigns: Game of Thrones,"October 18, 2018",50000 - 100000,3.99,84,815,698,203,Devolver Digital,"Adventure,Indie,RPG"
66,12140,Max Payne,"January 06, 2011",500000 - 1000000,3.49,89,8684,9516,1114,Rockstar Games,Action
...,...,...,...,...,...,...,...,...,...,...,...
97153,2305840,Cat Quest III,"August 08, 2024",50000 - 100000,19.99,84,1294,1403,77,Kepler Interactive,"Action,Adventure,Indie,RPG"
97235,2366980,Thank Goodness You're Here!,"August 01, 2024",50000 - 100000,19.99,90,2222,2244,67,Panic,"Adventure,Casual,Indie"
97248,2394650,Crypt Custodian,"August 27, 2024",0 - 20000,16.99,81,318,304,4,"Kyle Thompson,Top Hat Studios, Inc.,H2 Interac...",Adventure
97280,1299690,Gori: Cuddly Carnage,"August 29, 2024",0 - 20000,17.59,78,249,229,5,"Wired Productions,CouchPlay Interactive (Great...","Action,Adventure,Indie"


In [7]:
# Randomly sample 1000 rows from the dataset and save the result to a new file named dataset_test.csv
# Do not include the samples that are already in dataset_sample.csv

#df_filtered = df[~df["AppID"].isin(df_sample["AppID"])]
df_filtered = df
df_test = df_filtered.sample(n=600, replace=False)

df_test.to_csv("dataset_eval_t.csv", index=False)
df_test


,AppID,Name,Release date,Estimated owners,Price,Metacritic score,Recommendations,Positive,Negative,Publishers,Genres
56674,636230,Safe House,"May 22, 2018",0 - 20000,9.99,46,0,13,19,Labs Games,"Adventure,Indie,Strategy"
27639,256460,Cosmic Star Heroine,"April 11, 2017",100000 - 200000,14.99,77,574,677,82,Zeboyd Games,"Indie,RPG"
52131,327890,I Am Bread,"April 09, 2015",500000 - 1000000,12.99,60,4525,4359,1247,Bossa Studios,"Action,Adventure,Indie,Simulation"
35300,287020,Harvester,"April 04, 2014",50000 - 100000,5.99,53,769,905,88,Nightdive Studios,Adventure
54321,25980,Majesty 2,"September 17, 2009",200000 - 500000,9.99,72,783,611,270,Paradox Interactive,"Simulation,Strategy"
...,...,...,...,...,...,...,...,...,...,...,...
22468,269650,Dex,"May 07, 2015",200000 - 500000,19.99,62,2469,2655,389,"Dreadlocks Ltd.,Techland,WhisperGames","Action,Adventure,Indie,RPG"
50633,261900,The Real Texas,"July 12, 2016",0 - 20000,14.99,74,0,28,2,Kitty Lambda Games Inc.,"Action,Adventure,Indie,RPG"
37699,209630,Magrunner: Dark Pulse,"June 20, 2013",200000 - 500000,19.99,70,314,702,254,Frogwares,"Action,Adventure,Indie"
32579,578650,The Outer Worlds,"October 23, 2020",500000 - 1000000,19.79,82,17319,16925,2891,Private Division,RPG


In [8]:
# Randomly sample 5000 rows from the dataset and save the result to a new file named dataset_train.csv
# Do not include the samples that are already in other two files
df_filtered_train = df_filtered[~df_filtered["AppID"].isin(df_test["AppID"])]
df_train = df_filtered_train.sample(n=3000, replace=False)
df_train.to_csv("dataset_train.csv", index=False)

df_train

,AppID,Name,Release date,Estimated owners,Price,Metacritic score,Recommendations,Positive,Negative,Publishers,Genres
31648,350110,TransOcean 2: Rivals,"May 10, 2016",100000 - 200000,29.99,69,428,305,281,astragon Entertainment,"Simulation,Strategy"
34635,312780,Way of the Samurai 4,"July 23, 2015",200000 - 500000,24.99,72,1475,1288,318,"Spike Chunsoft Co., Ltd.","Action,Adventure"
54369,230150,Incredipede,"March 18, 2013",50000 - 100000,9.99,74,140,243,96,Northway Games,"Action,Adventure,Casual,Indie,Simulation"
41726,1252300,RetroMania Wrestling,"February 25, 2021",0 - 20000,29.99,66,174,211,30,Retrosoft Studios,"Action,Sports"
29574,760650,Hammerting,"November 16, 2021",100000 - 200000,9.99,64,1358,1328,705,Team17 Digital,"Adventure,Indie,RPG,Simulation,Strategy"
...,...,...,...,...,...,...,...,...,...,...,...
53872,266410,iRacing,"January 12, 2015",100000 - 200000,9.99,79,1281,2096,334,"iRacing.com Motorsport Simulations,iRacing","Massively Multiplayer,Racing,Simulation,Sports"
35601,939100,Darksburg,"September 23, 2020",100000 - 200000,14.99,61,1489,1511,750,Shiro Unlimited,"Action,Indie"
82625,1931770,Chants of Sennaar,"September 05, 2023",100000 - 200000,19.99,85,6125,6484,82,Focus Entertainment,"Adventure,Indie"
5602,41500,Torchlight,"October 27, 2009",1000000 - 2000000,14.99,83,4323,5424,495,Runic Games,RPG


In [11]:
# Load the dataset_eval_with_target.csv and drop the "Price" column and save the result to a new file named dataset_eval.csv
df_eval = pd.read_csv("dataset_eval_t.csv")
df_eval = df_eval.drop(columns=["Price"])
df_eval.to_csv("dataset_eval.csv", index=False)

df_eval

,AppID,Name,Release date,Estimated owners,Metacritic score,Recommendations,Positive,Negative,Publishers,Genres
0,636230,Safe House,"May 22, 2018",0 - 20000,46,0,13,19,Labs Games,"Adventure,Indie,Strategy"
1,256460,Cosmic Star Heroine,"April 11, 2017",100000 - 200000,77,574,677,82,Zeboyd Games,"Indie,RPG"
2,327890,I Am Bread,"April 09, 2015",500000 - 1000000,60,4525,4359,1247,Bossa Studios,"Action,Adventure,Indie,Simulation"
3,287020,Harvester,"April 04, 2014",50000 - 100000,53,769,905,88,Nightdive Studios,Adventure
4,25980,Majesty 2,"September 17, 2009",200000 - 500000,72,783,611,270,Paradox Interactive,"Simulation,Strategy"
...,...,...,...,...,...,...,...,...,...,...
595,269650,Dex,"May 07, 2015",200000 - 500000,62,2469,2655,389,"Dreadlocks Ltd.,Techland,WhisperGames","Action,Adventure,Indie,RPG"
596,261900,The Real Texas,"July 12, 2016",0 - 20000,74,0,28,2,Kitty Lambda Games Inc.,"Action,Adventure,Indie,RPG"
597,209630,Magrunner: Dark Pulse,"June 20, 2013",200000 - 500000,70,314,702,254,Frogwares,"Action,Adventure,Indie"
598,578650,The Outer Worlds,"October 23, 2020",500000 - 1000000,82,17319,16925,2891,Private Division,RPG


## Checking for null rows - don't set traps..

In [12]:
target_datasets = [df_train, df_eval]
for dfx in target_datasets:
    missing_rows = dfx[dfx.isna().any(axis=1)]
    assert len(missing_rows) == 0, "The datasets contain NULL tables"

missing_rows.head()

,AppID,Name,Release date,Estimated owners,Metacritic score,Recommendations,Positive,Negative,Publishers,Genres
